In [11]:
# pip3 install qiniu

In [1]:
import qiniu
from qiniu import BucketManager
import json
import requests
import os

In [2]:
access_key = os.popen('echo $qnpublic').read().strip()
secret_key = os.popen('echo $qnprivate').read().strip()
bucket_name = 'image'
bucket_url = os.popen('echo $qnimgurl').read().strip()

In [3]:
def qa():
    return qiniu.Auth(access_key,secret_key)

In [4]:
def token():
    q = qa()
    return q.upload_token(bucket=bucket_name)

In [5]:
def private_url(url):
    q = qa()
    return q.private_download_url(url,7200)

In [6]:
token = token()
print(token)

yXKy9jtDxW-WbSYdgeFiJmGFbLTeMqhgTOEUA7uc:ZQW2PaJ4cP5MlTnYhLURCbnJDzA=:eyJzY29wZSI6ImltYWdlIiwiZGVhZGxpbmUiOjE0NzEyNjE1MzR9


In [7]:
def file_list():
    bm = BucketManager(auth=qa())
    bm_rs = bm.list(bucket_name, None, None, 3, None)
    if bm_rs and bm_rs[0] and bm_rs[0].get('items') :
        items = bm_rs[0].get('items')
        print(items)
        return items

    return None

In [8]:
def img_url(items):
    img_url = []
    for it in items:
        img_url.append(bucket_url+it.get('key'))
    return img_url

In [9]:
items = file_list()
print(items)

[{'mimeType': 'image/jpeg', 'putTime': 14510979558388771, 'hash': 'Fmdq3jSqzCYrz8l4hVy5KHNDR6Ji', 'key': '0005e7e3-893a-4b67-9e2f-5f1dd7e9c5bb.jpg', 'fsize': 517791}, {'mimeType': 'image/jpeg', 'putTime': 14511222536092689, 'hash': 'FobVuAgxGMyv-xNcAkBT7sAZtsNp', 'key': '00082d4d-ae45-462b-82a5-5d3796dafaff.jpg', 'fsize': 63009}, {'mimeType': 'image/jpeg', 'putTime': 14511222973465557, 'hash': 'FgMzMHzwQUZbT9aNWCp32MdBHnPI', 'key': '001537e8-b1bc-4e27-8e43-981113d26d75.jpg', 'fsize': 30281}]
[{'mimeType': 'image/jpeg', 'putTime': 14510979558388771, 'hash': 'Fmdq3jSqzCYrz8l4hVy5KHNDR6Ji', 'key': '0005e7e3-893a-4b67-9e2f-5f1dd7e9c5bb.jpg', 'fsize': 517791}, {'mimeType': 'image/jpeg', 'putTime': 14511222536092689, 'hash': 'FobVuAgxGMyv-xNcAkBT7sAZtsNp', 'key': '00082d4d-ae45-462b-82a5-5d3796dafaff.jpg', 'fsize': 63009}, {'mimeType': 'image/jpeg', 'putTime': 14511222973465557, 'hash': 'FgMzMHzwQUZbT9aNWCp32MdBHnPI', 'key': '001537e8-b1bc-4e27-8e43-981113d26d75.jpg', 'fsize': 30281}]


In [10]:
imgs = img_url(items)
print(imgs)

['http://7xpbap.com1.z0.glb.clouddn.com/0005e7e3-893a-4b67-9e2f-5f1dd7e9c5bb.jpg', 'http://7xpbap.com1.z0.glb.clouddn.com/00082d4d-ae45-462b-82a5-5d3796dafaff.jpg', 'http://7xpbap.com1.z0.glb.clouddn.com/001537e8-b1bc-4e27-8e43-981113d26d75.jpg']


In [11]:
for i in imgs[:2] :
    print(private_url(i))

http://7xpbap.com1.z0.glb.clouddn.com/0005e7e3-893a-4b67-9e2f-5f1dd7e9c5bb.jpg?e=1471265139&token=yXKy9jtDxW-WbSYdgeFiJmGFbLTeMqhgTOEUA7uc:h-FFtUDnMqxVa6KZvx0o_4xaGqc=
http://7xpbap.com1.z0.glb.clouddn.com/00082d4d-ae45-462b-82a5-5d3796dafaff.jpg?e=1471265139&token=yXKy9jtDxW-WbSYdgeFiJmGFbLTeMqhgTOEUA7uc:Rv06A4URATWw4mRUfghQMWrQPxc=


In [12]:
'''
https://portal.qiniu.com/create/third_party_ufop/facecrop2/quickstart
'''
def facecrop(url):
    return url + '?facecrop2/300x300'
    

In [13]:
for i in imgs :
    print(facecrop(i))

http://7xpbap.com1.z0.glb.clouddn.com/0005e7e3-893a-4b67-9e2f-5f1dd7e9c5bb.jpg?facecrop2/300x300
http://7xpbap.com1.z0.glb.clouddn.com/00082d4d-ae45-462b-82a5-5d3796dafaff.jpg?facecrop2/300x300
http://7xpbap.com1.z0.glb.clouddn.com/001537e8-b1bc-4e27-8e43-981113d26d75.jpg?facecrop2/300x300


In [14]:
'''
https://portal.qiniu.com/create/third_party_ufop/tusdk-face/quickstart
ret ：返回码
message ：返回码说明
data ：true 为有人脸，false 为无人脸
ttp ：服务器时间戳
'''
def tusdk_face_check(url):
    url = url + '?tusdk-face/detection'
    r = requests.get(url)
    print(r.status_code)
    if r.status_code == 200 :
        print(r.json())
        js = r.json()
        return js['data']

In [15]:
'''
响应内容

ret ：返回码
message ：返回码说明
data ：count 为人脸数，items 为人脸特征点数据
ttp ：服务器时间戳
'''
def tusdk_face_landmark(url):
    if tusdk_face_check(url):
        url = url + '?tusdk-face/landmark'
        r = requests.get(url)
        print(r.status_code)
        if r.status_code == 200 and r.json()['ret'] == 200:
            print(r.json())
#             js = r.json()
#             return js['data']
        

In [16]:
for i in imgs[:2] :
    print(tusdk_face_landmark(i))

200
{'data': False, 'ret': -601, 'message': 'No face is detected', 'ttp': 1471070551}
None
200
{'data': True, 'ret': 200, 'message': 'Success', 'ttp': 1471070571}
200
{'data': {'items': [{'faceId': '401d722f71a45d253f5eece335e92cab', 'marks': {'right_brow_upper_middle': {'y': 0.271885, 'x': 0.342613}, 'nose_contour_right_corner': {'y': 0.354143, 'x': 0.324615}, 'mouth_lower_outer_contour3': {'y': 0.391239, 'x': 0.317733}, 'nose_top': {'y': 0.301493, 'x': 0.315574}, 'mouth_upper_outer_contour1': {'y': 0.368833, 'x': 0.307938}, 'contour_right4': {'y': 0.337306, 'x': 0.37417}, 'contour_right5': {'y': 0.305368, 'x': 0.376082}, 'mouth_lower_inner_contour2': {'y': 0.380764, 'x': 0.317347}, 'right_brow_lower_middle': {'y': 0.276529, 'x': 0.342885}, 'contour_left4': {'y': 0.392067, 'x': 0.297666}, 'nose_contour_left_corner': {'y': 0.353962, 'x': 0.306933}, 'right_eye_bottom_left': {'y': 0.304704, 'x': 0.336472}, 'left_brow_upper_middle': {'y': 0.272999, 'x': 0.295638}, 'left_eye_bottom_middle'